# **fine tuning** with IMDB DATASET

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
dataset = load_dataset('imdb')

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [5]:
# Preprocess the Data
##Tokenize the dataset using the tokenizer associated with the pre-trained model.

from transformers import AutoTokenizer

# Load tokenizerr
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenized dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [7]:
tokenized_datasets["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

# Set Up the Training Arguments

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy ="epoch",     # Evaluate every epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=1,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
)
training_args

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    evaluation_strategy="no",          # Skip evaluation
    learning_rate=2e-5,
    per_device_train_batch_size=8,     # Smaller batch size to reduce memory usage
    num_train_epochs=0.3,              # Fewer epochs
    weight_decay=0.01,
    logging_steps=500,
    save_strategy="no",                # Skip saving model checkpoints
    fp16=True                          # Use half-precision if on supported GPU
)


c:\Users\deepa\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
#Initialize the Model

from transformers import AutoModelForSequenceClassification, Trainer

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: deepak-maurya-kapisoorr (deepak-maurya-kapisoorr-kapisoorr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 0/938 [00:00<?, ?it/s]

In [ ]:
# prompt: Assess the model's performance on a validation set.

eval_results = trainer.evaluate()
eval_results                       # evalation loss


{'eval_loss': 0.2469930499792099,
 'eval_runtime': 192.0292,
 'eval_samples_per_second': 130.189,
 'eval_steps_per_second': 16.274,
 'epoch': 0.30016}

In [ ]:
# Save the model
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-tokenizer')


NameError: name 'model' is not defined

In [ ]:
from transformers import pipeline

# Load the model
pipe = pipeline("text-classification", model="/content/fine-tuned-model", tokenizer="/content/fine-tuned-tokenizer")

# Predict
pipe("movies is not bad")  # -> Output: [{'label': 'POSITIVE', 'score': 0.98}]


Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.6206133365631104}]

In [ ]:
from transformers import AutoModelForSequenceClassification

labels = {"negative": 0, "positive": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "/content/fine-tuned-model",
    num_labels=2,
    id2label={v: k for k, v in labels.items()},
    label2id=labels
)

print(labels.items())

dict_items([('negative', 0), ('positive', 1)])


In [ ]:
pipe = pipeline("text-classification", model=model, tokenizer="/content/fine-tuned-tokenizer")
pipe("movies is not bad")

Device set to use cuda:0


[{'label': 'negative', 'score': 0.6206133365631104}]

Mounted at /content/drive
